In [ ]:
# faire les importations nécessaires
import psycopg2
import google.generativeai as genai
from psycopg2 import sql
import numpy as np

# Déclarer les variables nécessaires
conversation_file_path = "..\data\DISTRIBUTION_ACCUEIL_UBS\TRANS_TXT\018_00000013.txt" 
print(f"Chemin du fichier de conversation: {conversation_file_path}")
model = "models/embedding-001"  # Modèle d'embedding de Gemini

# Initialiser le client Gemini
genai.configure(api_key="")

db_connection_str = "dbname=rag_chatbot user=postgres password=tasnim host=localhost port=5432"

def create_conversation_list(file_path: str) -> list[str]:
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
            text_list = text.split("\n")
            filtered_list = [chaine.strip() for chaine in text_list if chaine.strip() and not chaine.startswith("<")]
            print(f"Nombre de conversations chargées: {len(filtered_list)}")
            return filtered_list
    except FileNotFoundError:
        print(f"Fichier non trouvé: {file_path}")
        return []
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {e}")
        return []

def calculate_embeddings(corpus: str) -> list[float]:
    """Calcule les embeddings avec Gemini"""
    try:
        result = genai.embed_content(
            model=model,
            content=corpus,
            task_type="retrieval_document"
        )
        return result['embedding']
    except Exception as e:
        print(f"Erreur lors du calcul de l'embedding: {e}")
        return []

def setup_database(db_connection_str: str) -> bool:
    """Configure la base de données et retourne True si pgvector est disponible"""
    try:
        with psycopg2.connect(db_connection_str) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                # Essayer de créer l'extension pgvector
                try:
                    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
                    print("✓ Extension pgvector activée")
                    return True
                except Exception as e:
                    print(f"✗ pgvector non disponible: {e}")
                    print("✓ Utilisation du fallback avec FLOAT8[]")
                    return False
    except Exception as e:
        print(f"Erreur de connexion à la base: {e}")
        return False

def create_table_with_fallback(db_connection_str: str, use_vector: bool):
    """Crée la table avec le type approprié"""
    with psycopg2.connect(db_connection_str) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute("DROP TABLE IF EXISTS embeddings;")
            
            if use_vector:
                # Avec pgvector
                create_table_query = """
                CREATE TABLE IF NOT EXISTS embeddings (
                    ID SERIAL PRIMARY KEY, 
                    corpus TEXT,
                    embedding VECTOR(768)
                );
                """
            else:
                # Fallback sans pgvector
                create_table_query = """
                CREATE TABLE IF NOT EXISTS embeddings (
                    ID SERIAL PRIMARY KEY, 
                    corpus TEXT,
                    embedding FLOAT8[]
                );
                """
            
            cur.execute(create_table_query)
            print("✓ Table embeddings créée")

def save_embedding(corpus: str, embedding: list[float], cursor, use_vector: bool) -> None:
    """Sauvegarde l'embedding dans la base de données"""
    try:
        if use_vector:
            # Pour pgvector
            embedding_array = "[" + ",".join(str(x) for x in embedding) + "]"
            cursor.execute(
                '''INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)''', 
                (corpus, embedding_array)
            )
        else:
            # Pour FLOAT8[]
            embedding_array = "{" + ",".join(str(x) for x in embedding) + "}"
            cursor.execute(
                '''INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)''', 
                (corpus, embedding_array)
            )
    except Exception as e:
        print(f"Erreur lors de la sauvegarde: {e}")

def similar_corpus(input_corpus: str, db_connection_str: str, use_vector: bool, top_k: int = 5) -> list[tuple]:
    """
    Trouve les textes similaires dans la base de données
    """
    input_embedding = calculate_embeddings(input_corpus)
    if not input_embedding:
        print("Erreur: Impossible de calculer l'embedding de la requête")
        return []
    
    try:
        with psycopg2.connect(db_connection_str) as conn:
            with conn.cursor() as cur:
                if use_vector:
                    # Requête avec pgvector
                    input_embedding_array = "[" + ",".join(str(x) for x in input_embedding) + "]"
                    query = """
                    SELECT id, corpus, 
                           embedding <=> %s::vector AS distance
                    FROM embeddings 
                    ORDER BY embedding <=> %s::vector
                    LIMIT %s
                    """
                else:
                    # Requête sans pgvector (similarité cosinus manuelle)
                    input_embedding_array = "{" + ",".join(str(x) for x in input_embedding) + "}"
                    query = """
                    SELECT id, corpus,
                           sqrt(power(embedding - %s, 2)) as distance
                    FROM embeddings 
                    ORDER BY distance
                    LIMIT %s
                    """
                
                if use_vector:
                    cur.execute(query, (input_embedding_array, input_embedding_array, top_k))
                else:
                    cur.execute(query, (input_embedding_array, top_k))
                
                results = cur.fetchall()
                
                print(f"Top {len(results)} résultats similaires:")
                for i, (id, corpus, distance) in enumerate(results, 1):
                    print(f"{i}. ID: {id}, Distance: {distance:.4f}")
                    print(f"   Texte: {corpus[:100]}...")
                    print("-" * 80)
                
                return results
    except Exception as e:
        print(f"Erreur lors de la recherche: {e}")
        return []

# Exécution principale
print("Démarrage du système RAG...")

# Étape 1: Configuration de la base
use_vector = setup_database(db_connection_str)

# Étape 2: Création de la table
create_table_with_fallback(db_connection_str, use_vector)

# Étape 3: Chargement et traitement des données
corpus_list = create_conversation_list(file_path=conversation_file_path)

if corpus_list:
    with psycopg2.connect(db_connection_str) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            for i, corpus in enumerate(corpus_list):
                if corpus.strip() and len(corpus) > 10:  # Ignorer les textes trop courts
                    print(f"Traitement du corpus {i+1}/{len(corpus_list)}: {corpus[:50]}...")
                    embedding = calculate_embeddings(corpus=corpus)
                    if embedding:
                        save_embedding(corpus=corpus, embedding=embedding, cursor=cur, use_vector=use_vector)
            
            conn.commit()
            print("✓ Tous les embeddings ont été sauvegardés")

    # Étape 4: Test de recherche
    print("\n" + "="*50)
    print("TEST DE RECHERCHE PAR SIMILARITÉ")
    print("="*50)

    test_queries = [
        "Bonjour, comment puis-je vous aider ?",
        "Je cherche des informations",
        "Problème avec mon compte"
    ]

    for test_query in test_queries:
        print(f"\nRecherche pour: '{test_query}'")
        similar_results = similar_corpus(
            input_corpus=test_query, 
            db_connection_str=db_connection_str,
            use_vector=use_vector,
            top_k=3
        )
else:
    print("Aucune donnée à traiter")

Chemin du fichier de conversation: ..\data\DISTRIBUTION_ACCUEIL_UBS\TRANS_TXT8_00000013.txt
Démarrage du système RAG...
✗ pgvector non disponible: ERREUR:  l'extension « vector » n'est pas disponible
HINT:  Les extensions doivent tout d'abord être installées sur le système où PostgreSQL est exécuté.

✓ Utilisation du fallback avec FLOAT8[]
✓ Table embeddings créée
Erreur lors de la lecture du fichier: [Errno 22] Invalid argument: '..\\data\\DISTRIBUTION_ACCUEIL_UBS\\TRANS_TXT\x018_00000013.txt'
Aucune donnée à traiter
